# Aula 4 - Regressão Logística

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Introdução
- 2) Regressão logística
- 4) Métricas de performance para problemas de classificação

____
____
____

## 1) Introdução

**Problemas de classificação** são aqueles em que queremos determinar a que **categoria ou classe** dentro de um **conjunto discreto de classes** uma dada observação pertence, com base em suas features.

Para isso, construímos um **classificador**: modelo que tem como input as features (contínuas ou discretas) e como output uma entre as classes (discretas).

> Principal diferença entre problemas de regressão e classificação:
> - Regressão: valores contínuos;
> - Classificação: valores (classes) discretas (binárias ou não).

No caso de regressão, a hipótese será a equação que determina o target diretamente;

No caso de classificação, a hipótese tem o objetivo de **separar as diferentes classes**. Por isso, muitas vezes utilizamos o termo **fonteira de classificação**, ou **fronteira de decisão**. De um lado da fronteira, temos uma classe; do outro, a outra classe.

<img src="https://i0.wp.com/vinodsblog.com/wp-content/uploads/2018/11/Classification-vs-Regression.png?fit=2048%2C1158&ssl=1" width=700>

<img src="https://i.pinimg.com/originals/71/8e/6a/718e6a40e1782bead960e58d3c52663b.png" width=300>

Problemas de classificação são comumente divididos com relação ao **número de classes** a serem preditas (isto é, com relação à estrutura do espaço de target):

- Classificação binária: duas classes (0 e 1);
- Classificação multiclasse: $n$ classes (0, 1, ..., $n-1$), com $n > 2 \in \mathbb{N}$

Exemplos de problemas de classificação:
- Detecção de e-mails SPAM: um e-mail é SPAM ou não?;
    - Features: palavras contidas no corpo do e-mail; remetente; assunto;
- Detecção de doenças: que codição médica a pessoa tem?
    - Features: sintomas fisiológicos; resultados de exames (medidas de variáveis biológicas);
- Detecção do tipo de documento: secreto, confidencial ou não-sensível?
    - Features: palavras no corpo do texto; título;
- Detecção de fraudes de cartão de crédito: uma operação é fraudulenta ou não?;
    - Features: histórico de transações; hora, local e frequência das transações; tipo de compra;
- Modelo de risco de crédito: qual é a chance de determinada pessoa não pagar seu empréstimo?
    - Features: histórico de pagamento; score de crédito;
    
    
<img src="https://developers.google.com/machine-learning/guides/text-classification/images/TextClassificationExample.png" width=500>



Veremos hoje um dos mais simples e importantes classificadores: a **Regressão Logística!**

Antes de conhecermos o método, vamos dar uma rápida olhada na base qual a qual trabalharemos hoje!

__________________

Para introduzirmos as ideias, utilizaremos um dataset de marketing (propagandas/advertising), que está disponível no <a href="https://www.kaggle.com/fayomi/advertising">Kaggle</a>. Este é um dataset artificial e didático, com os dados bem separáveis, o que é ótimo para ilustração!<br>

Visite o Kaggle e procure por "advertising" para datasets relacionados reais e ainda mais interessantes

A base que utilizaremos contém as seguintes colunas:

* 'Daily Time Spent on Site': tempo que o cliente ficou no site (em minutos);
* 'Age': idade do cliente (em anos);
* 'Area Income': média salarial (por ano) da região geográfica do cliente;
* 'Daily Internet Usage': tempo médio (em minutos) que o cliente fica na internet;
* 'Ad Topic Line': título do anúncio;
* 'City': cidade do cliente;
* 'Male': dummy indicando se o cliente é do sexo masculino (1) ou não (0);
* 'Country': país do cliente;
* 'Timestamp': marcação de tempo em que o cliente clickou no anúncio OU fechou a página
* 'Clicked on Ad': dummy indicando se o cliente clickou no anúncio (1) ou não (0).

Nosso objetivo é criar um modelo que possa prever se um determinado usuário clickará em um anúncio online ou não, com base em suas características pessoais/comportamentais, bem como informações relativas ao anúncio.

Tomamos como variáveis independentes (preditores/features) as primeiras 9 colunas, enquanto nossa variável dependente (target) é a última coluna ("Clicked on Ad").

Ou seja, nosso modelo deve ser capaz de dizer se um usuário com um conjunto particular das 9 features clickará no anúncio ou não. 

__IMPORTANTE!__

Pense no problema de negócio que estamos querendo resolver com nosso modelo -- direcionamento de marketing! Temos os dados dos nossos clientes (customer-centric), nós os conhecemos! Não podemos utilizar essa informação a nosso favor?

Talvez não faça sentido exibir o anúncio para um usuário que tem baixa probabilidade de clickar no ad, não é mesmo? 

Por outro lado, é muito mais eficiente direcionar nosso marketing aos clientes com alta chance de clickar no nosso anúncio!

Assim, economizamos dinheiro (todo anúncio é pago!), e ganhamos em eficiência e alcance!

___
___
___

In [ ]:
# imports

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

O código abaixo é apenas para formatar os números em até 3 casas decimais. 

Fica aqui pra conhecimento e também pq vai nos auxiliar a ver melhor as probabilidades no final.

In [ ]:
pd.set_option("display.float_format", lambda x: "%.3f" % x)

np.set_printoptions(suppress=True, precision=3)

Vamos ler a base!

In [ ]:
df = pd.read_csv("../datasets/advertising.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["Clicked on Ad"].unique()

### TODO: EDA em casa

__________

Agora que conhecemos brevemente o problema, vamos conhecer o método de modelagem!

> Nossa discussão será feita toda em cima do problema de **classificação binária**, ou seja, com o espaço de target $\mathcal{Y} = \{0, 1\}$ (as labels 0 e 1 são arbitrárias, e simplesmente diferenciam os dois valores possíveis para o target. Uma outra codificação comum é $\mathcal{Y} = \{-1, +1\}$)

Para o caso multiclasse, há algumas anterações nos fundamentos dos métodos, mas, na prática, a implementação será direta, então não precisamos nos preocupar!

___
___
___

## 2) Regressão Logística

A [Regressão Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) (também chamado de **logit**), apesar do nome, é um método que aplicaremos a problemas de classificação!

O objetivo da regressão logística é: **modelar a probabilidade $P(\vec{x})$ de dada observação (com features $\vec{x}$) pertencer a uma das classes (que comumente chamamos de classe 1)**, ou seja, queremos modelar:

$$ P( y = 1 | \vec{x}) $$

Naturalmente, $0 \le P(\vec{x}) \le 1$. 

> Lembre-se que: $ P( y = 0 | \vec{x}) = 1 - P( y = 1 | \vec{x}) $

Uma vez que tivermos uma função que modele a probabilidade acima, podemos tomar a decisão de classificação da seguinte maneira:

- $P(\vec{x}) \ge 0,5$: x pertence à classe 1
- $P(\vec{x}) < 0.5$: x pertence à classe 0

Obs.: este valor de 0.5 (50%) é chamado de "cutoff", e pode ser ajustado, embora o default fixá-lo em 50%!

> É justamente através do cutoff que tomamos uma decisão discreta (classificação) a partir de um método de regressão!

Poderíamos pensar em utilizar a regressão linear em nossos problemas de classificação, mas isso não é uma boa ideia: acabamos encontrando probabilidades negativas e fit ruim!

No exemplo a seguir, temos a probabilidade de não-pagamento (default) de um empréstimo com base em uma feature (balanço). Note probabilidades negativas!

<figure>
    <img src="https://s3-sa-east-1.amazonaws.com/lcpi/70189f79-2886-4e59-893b-1dac9dd64078.png" height="400" width="400">
</figure> 

Para resolver este problema, podemos adaptar a função de regressão linear para uma função que tem imagem entre 0 e 1. Seria legal se tivéssemos algo como:

<figure>
    <img src="https://s3-sa-east-1.amazonaws.com/lcpi/6d54529a-d295-47a3-8a11-1f426fde7229.png" height="400" width="400">
</figure> 

Um exemplo de tal função é a **função logística** ou **função sigmoidal**:

<img src="https://miro.medium.com/max/970/1*Xu7B5y9gp0iL5ooBj7LtWw.png" width=400>

Note que:

- $z \in \mathbb{R}$
- $0 \le \phi(z) \le 1$

Para incorporar a ideia da regressão linear na regressão logística, tomamos:

- $z = b_0 + b_1x$, que é o modelo de regressão linear (uma variável);

E substituímos na função logística:

- $\phi(x) = \frac{1}{1 + e^{-(b_0 + b_1 x)}}$

Com isso, tomamos qualquer output real do modelo linear e transformamos em um valor entre 0 e 1, como queríamos!

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/e5ecf372-6790-49db-9bad-95bc4b19df27.png" width="500">

No nosso caso, como queremos modelar probabilidades, a função acima é exatamente a **hipótese** do estimador de regressão logística! Isto é,

$$f_{H, \vec{b}}(x) = P(x) = \frac{1}{1 + e^{-(b_0 + b_1 x)}}$$

Ou, para a regressão logística múltipla com $p$ features $\vec{x} = x_1, \cdots, x_p$:

$$f_{H, \vec{b}}(\vec{x}) = P(\vec{x}) = \frac{1}{1 + e^{-(b_0 + b_1 x_1 + \cdots + b_p x_p)}}$$

No fim, temos que a predição é tomada em termos do cutoff $\alpha$, e, com isso, chegamos no $\hat{y}$:

$$\hat{y} = \left\{\begin{matrix}
1, \text{se } P(\vec{x}) \geq \alpha\\  
0, \text{se } P(\vec{x}) < \alpha
\end{matrix}\right.$$

Note, portanto, que apesar da hipótese ser uma função não linear, **a fronteira de decisão** é linear, sendo definida pela função por partes acima, com base no cutoff e na probabilidade!

Com um pouco de álgebra, é possível mostrar que: 

$ b_0 + b_1 x_1 + \cdots + b_p x_p = \log \left ( \frac{P}{1-P} \right ) $

A quantidade $\frac{P}{1-P}$ é conhecida como **odds/chance**; e $\log \left ( \frac{P}{1-P} \right )$ é o [log-odds ou logit](https://en.wikipedia.org/wiki/Logit).

Note, portanto, que podemos entender a regressão logística como um modelo em que **o logit é linear com as features**. 

> Portanto, esse fato e o anterior fazem com que, de fato, a regressão logística seja um **um modelo linear**.

Na regressão logística, nosso conjunto de hipóteses é: $\mathcal{H} = \left \{ \frac{1}{1 + e^{-(b_0 + b_1 x_1 + \cdots + b_p x_p)}} \right \}$.

O objetivo do algoritmo de aprendizagem será, como sempre, determinar qual é o vetor de parâmetros $\vec{b}$ que produz uma função logística que **melhor se ajusta aos dados**.

Para ilustrar este ponto novamente, vamos produzir a seguir algumas das infinitas funções de $\mathcal{H}$:

In [ ]:
def hip_lin(x, b0, b1):
    
    return b0 + b1*x

In [ ]:
def sig(x):
    
    return 1/(1 + np.exp(-x))

In [ ]:
b0 = 1
b1 = 2

x = np.linspace(-10, 10, 1000)

y_lin = hip_lin(x, b0, b1)

y_sig = sig(y_lin)

plt.plot(x, y_sig)

In [ ]:
b0 = 1
b1 = 2

x = np.linspace(-10, 10, 1000)

# composição de funções! 
y = sig(hip_lin(x, b0, b1))

plt.plot(x, y)

In [ ]:
x = np.linspace(-50, 50, 1000)

b0 = 1
b1_list = [0, -0.1, -0.5, 0.5, 0.1]

for b1 in b1_list:

    # composição de funções! 
    y = sig(hip_lin(x, b0, b1))

    plt.plot(x, y, label=f"b1={b1}")
    
plt.legend()
plt.show()

In [ ]:
x = np.linspace(-10, 10, 1000)

b1 = 1
b0_list = [0, -2, -0.5, 0.5, 2]

for b0 in b0_list:

    # composição de funções! 
    y = sig(hip_lin(x, b0, b1))

    plt.plot(x, y, label=f"b0={b0}")
    
plt.legend()
plt.show()

Com a biblioteca [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) podemos fazer plots interativos super legais!

Valeu Marcelo e Alexandre pela ótima dica!!

In [ ]:
import ipywidgets as widgets

def reg_lod_widget(b1=1, b0=1):
    
    # composição de funções! 
    y = sig(hip_lin(x, b0, b1))

    plt.plot(x, y, label=f"$b_0$={b0} | $b_1$={b1}")
    
    plt.legend()
    
    plt.ylim(-0.25, 1.25)

In [ ]:
widgets.interact(reg_lod_widget, b0=(-2, 2, 0.1), b1=(-2, 2, 0.1));

___

### Função de perda e algoritmo de aprendizagem

A função de perda para a regressão logística é a famosa [binary cross-entropy](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a), também conhecida como [log loss](https://developers.google.com/machine-learning/crash-course/logistic-regression/model-training)

Esta função será de enorme importância no estudo de **redes neurais**.

As principais implementações do algoritmo de aprendizagem da regressão logística se baseia no [método de máxima verossimilhança](https://pt.wikipedia.org/wiki/M%C3%A1xima_verossimilhan%C3%A7a). 

Para maiores detalhes sobre o algoritmo de aprendizagem, veja [este vídeo](https://youtu.be/yIYKR4sgzI8) e [esta série de vídeos](https://youtu.be/vN5cNN2-HWE), do ótimo canal StatQuest!


_________

Vamos analisar um pouco mais nosso dataset de marketing...

In [ ]:
df

Aqui novamente, vamos considerar apenas as colunas numéricas como features. 

Sigamos com o train-test split!

In [ ]:
df_model = df.select_dtypes(include=np.number)

In [ ]:
X = df_model.drop(columns="Clicked on Ad")
y = df_model["Clicked on Ad"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
for col in X_train:
    
    sns.histplot(data=X_train, x=col, kde=True)
    
    plt.show()

Quando fizermos a EDA de um problema de classificação, é muito útil utilizar o target para analisar a **separabilidade** das classes! 

Para este fim, basta usarmos o argumento `hue` das funções do seaborn!

In [ ]:
for col in X_train:
    
    sns.histplot(data=X_train, x=col, kde=True, hue=y_train)
    
    plt.show()

O `pairplot` é uma ferramente legal para visualizarmos nossos dados projetados ao subspaço de cada par de features:

In [ ]:
pd.concat([X_train, y_train], axis=1)

In [ ]:
sns.pairplot(data=pd.concat([X_train, y_train], axis=1), hue="Clicked on Ad")

Como tínhamos comentado no início, nossos dados são muito bem separáveis!

Isto favorece bastante a performance do nosso modelo. Mas, lembre-se, é bem raro encontrar casos assim na vida real! (É aí que devemos partir para métodos mais avançados, como SVM, árvores, etc.)

Vamos começar a construir o modelo?

In [ ]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression().fit(X_train, y_train)

__Modelo treinado!__

$$f_{H, \vec{b}}(\vec{x}) = P(y=1 | \vec{x}) = \frac{1}{1 + e^{-(b_0 + b_1 x_1 + \cdots + b_p x_p)}}$$

Vamos ver os parâmetros do modelo:

In [ ]:
logit.intercept_

In [ ]:
logit.feature_names_in_

In [ ]:
logit.coef_

In [ ]:
logit.coef_[0][2]

Lembre-se que, diferentemente da regressão linear, devido ao fato da função logística ser uma exponencial, a variação de $P(x)$ depende de x, e não apenas dos coeficientes! Então, a interpretação dos coeficientes não é tão imediata. 

Mas, os sinais carregam significado. Para um coeficiente:
- positivo ($b_i > 0$), temos que um aumento em x levará a um aumento de $P(x)$;
- negativo ($b_i < 0$), temos que um aumento em x levará a uma diminuição de $P(x)$

Mas, a variacão de $P(x)$ em si, depende do valor de x!

__Agora que o modelo está treinado, vamos avaliá-lo!__

______
_____
_____
____

In [ ]:
X_test

In [ ]:
y_test.values

In [ ]:
logit.predict(X_test)

Além dos coeficientes do modelo, algo muito interessante que a classe do sklearn proporciona é o método `predict_proba()`

Esse método retorna exatamente qual é a **probabilidade modelada pelo logit**, isto é, $P(y=1 | \vec{x})$.

Isso pode ser muito útil, pois assim conseguimos **mudar qual é o cutoff de escolha de classe** para ser algo diferente de 0.5!


In [ ]:
logit.classes_

In [ ]:
proba_1 = logit.predict_proba(X_test)[:, 1]

proba_1

In [ ]:
cutoff = 0.1

np.where(proba_1 >= cutoff, 1, 0)

In [ ]:
cutoff = 0.9

np.where(proba_1 >= cutoff, 1, 0)

## 3) Métricas de performance para problemas de classificação

Após treinar o modelo, como podemos avaliar sua performance?

No caso de problemas de classificação, existem **métricas específicas**, e também um importante conceito chamado de **Matriz de Confusão**.

A **matriz de confusão** leva em consideração as **classes preditas** e as **classes verdadeiras** da base de **teste**, e contabiliza a performance do modelo:

<img src=https://diegonogare.net/wp-content/uploads/2020/04/matrizConfusao-600x381.png height="400" width="400">

Note que a diagonal principal são as observações que o modelo acertou! Temos:

- Verdadeiros Positivos (VP): classificação correta da classe positivo;
- Verdadeiros Negativos (VN): classificação correta da classe negativo;
- Falsos Positivos (FP, erro tipo I): correto: negativo. Previsto: positivo.
- Falsos Negativos (FN, erro tipo II): correto: positivo. Previsto: negativo.

Um jeito fácil de lembrar os tipos de erros:

<img src="https://i.pinimg.com/originals/f6/9b/11/f69b111014ef466fe541a393346d2c3a.jpg" height="400" width="400">

> **IMPORTANTE**: dependendo da implementação/referência, a ordem das linhas/colunas pode mudar, então se atente a isso quando for interpretar a matriz de confusão!

No Sklearn, a convenção é a seguinte:

<img src="https://static.packt-cdn.com/products/9781838555078/graphics/C13314_06_05.jpg" width=400>

Além disso, temos as seguintes métricas numéricas de avaliação:

- Acurácia (Accuracy): porcentagem de classificações CORRETAS do modelo;

- Precisão (Precision): das respostas retornadas, quantas são relevantes? -- é a razão entre verdadeiros positivos e o  número de **preditos positivos**, isto é, positivos quanto à **label predita pelo modelo**.

- Revocação/Sensibilidade (Recall/Sensitivity): das respostas relevantes, quantas são retornadas? -- é a razão entre verdadeiros positivos e o  número de **verdadeiramente positivos**, isto é, positivos quanto à **label real**.

- F1-Score: média harmônica de precision e recall.

Descrição da imagem: 

> tudo o que tá no lado esquerdo é a classe real positiva (y = 1); do lado direito, real negativa (y = 0);

> tudo o que tá dentro do circulo predita positiva ($\hat{y} = 1$); fora do circulo, predita negativa ($\hat{y} =0$)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/1200px-Precisionrecall.svg.png" width=400>

Devido ao <a href="https://medium.com/opex-analytics/why-you-need-to-understand-the-trade-off-between-precision-and-recall-525a33919942">tradeoff entre precision e recall</a>, uma métrica que em muitos casos é interessante de ser otimizada é o F1! 

<img src="https://miro.medium.com/max/1080/1*t1vf-ofJrJqtmam0KSn3EQ.png" width=500>

Adiante, veremos como calcular a matriz de confusão e as métricas acima para problemas de classificação!

_______

Um ponto muito importante é que o método `predict()` se utiliza do cutoff igual a 0.5 para tomar a decisão! Veremos mais detalhes sobre isso mais a frente. Por enquanto, vamos seguir com a avaliação do modelo com este cutoff padrão!

In [ ]:
# y_pred = logit.predict(X_test)

proba_1 = logit.predict_proba(X_test)[:, 1]
cutoff = 0.5
y_pred = np.where(proba_1 >= cutoff, 1, 0)

y_pred

Como vimos no passo 2, em problemas de classificação é muito comum utilizarmos a **matriz de confusão** e as **métricas de classificação** para avaliar nossos modelos.

Dado isso, o sklearn já disponibilica estas funcionalidades! Vejamos algumas delas!

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize="all")

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
cr_dict = classification_report(y_test, y_pred, output_dict=True)

In [ ]:
cr_dict

In [ ]:
cr_dict["1"]

In [ ]:
cr_dict["1"]["precision"]

In [ ]:
cr_dict["weighted avg"]

Conforme esperado, nosso modelo está muito bom! Um f1-score tão alto na vida real é algo notável!

Isso se deve à grande separabilidade dos nossos dados!

In [ ]:
def clf_metrics(modelo, X, y, cutoff=0.5, label_metrica="", print_plot=True):
    
    proba_1 = logit.predict_proba(X)[:, 1]
    y_pred = np.where(proba_1 >= cutoff, 1, 0)
    
    if print_plot:
        
        print(f"Métricas de avaliação de {label_metrica}")
    
        ConfusionMatrixDisplay.from_predictions(y, y_pred)
        plt.show()
    
        print(classification_report(y, y_pred))
    
    return classification_report(y, y_pred, output_dict=True)

In [ ]:
cr_train = clf_metrics(logit, X_train, y_train, label_metrica="treino", cutoff=0.5)

In [ ]:
y_test.shape

In [ ]:
cr_test = clf_metrics(logit, X_test, y_test, label_metrica="teste", cutoff=0.5)

In [ ]:
cr_test

_________________

Vamos avaliar diferentes cutoffs...

In [ ]:
logit

Pra entender o código abaixo

In [ ]:
df_test = pd.DataFrame(cr_test)

df_test

In [ ]:
df_train = pd.DataFrame(cr_train)

df_train

In [ ]:
results_train = {"cutoff" : []}

# o [:-1] é pra não trazer o "support" (se quiser trazer, é só tirar isso)
for idx in df_train.index[:-1]:
    
    for col in df_train:
        
        results_train[f"{idx}_{col}"] = []
        
results_train

In [ ]:
results_test = {"cutoff" : []}

for idx in df_test.index[:-1]:
    
    for col in df_test:
        
        results_test[f"{idx}_{col}"] = []
        
results_test

In [ ]:
cutoff_list = np.arange(0.1, 1, 0.05)

for cutoff in cutoff_list:
    
    print()
    print("#"*80)
    print(f"Modelo logit com cutoff = {cutoff}".center(80))
    print("#"*80)
    print()
    
    # ============================================
    
    cr_train = clf_metrics(logit, X_train, y_train, label_metrica="treino", cutoff=cutoff, print_plot=False)
    df_cr_train = pd.DataFrame(cr_train)

    results_train["cutoff"].append(cutoff)

    for idx in df_cr_train.index[:-1]:

        for col in df_cr_train:

            results_train[f"{idx}_{col}"].append(df_cr_train.loc[idx, col])
            
    # ============================================
    
    cr_test = clf_metrics(logit, X_test, y_test, label_metrica="teste", cutoff=cutoff)
    df_cr_test = pd.DataFrame(cr_test)
    
    results_test["cutoff"].append(cutoff)
    
    for idx in df_cr_test.index[:-1]:

        for col in df_cr_test:

            results_test[f"{idx}_{col}"].append(df_cr_test.loc[idx, col])
    
    # ============================================

In [ ]:
df_results_train = pd.DataFrame(results_train)

df_results_train.sort_values("precision_1", ascending=False)

In [ ]:
df_results_test = pd.DataFrame(results_test)

df_results_test.sort_values("precision_1", ascending=False)

Alternativamente, podemos fazer como abaixo (adaptado da sugestão do Marcelo, créditos a ele!)

In [ ]:
# começamos com dois dfs vazios, e vamos preenchendo abaixo
# (com os concats)
df_results_train = pd.DataFrame()
df_results_test = pd.DataFrame()

cutoff_list = np.arange(0.1, 1, 0.05)

for cutoff in cutoff_list:
    
    print()
    print("#"*80)
    print(f"Modelo logit com cutoff = {cutoff}".center(80))
    print("#"*80)
    print()
    
    # ============================================
    
    cr_train = clf_metrics(logit, X_train, y_train, label_metrica="treino", cutoff=cutoff, print_plot=False)
    df_cr_train = pd.DataFrame(cr_train)

    # o .iloc[:-1, :] é pra excluir o "support" (pode trazer tb caso queira)
    df_melt = df_cr_train.iloc[:-1, :].reset_index(drop=False).melt(id_vars='index', var_name="type")
    df_melt['cutoff'] = cutoff

    # corrigindo os dados de acurácia
    accuracy = df_melt.query("type == 'accuracy'").copy()
    accuracy["index"] = "accuracy"

    df_melt = df_melt.drop(index=accuracy.index)
    df_melt = pd.concat([df_melt, accuracy.iloc[[0], :]]).reset_index(drop=False)

    # pivotando
    df_pivot = df_melt.pivot(columns=['type', 'index'], values='value', index='cutoff')
    
    df_results_train = pd.concat([df_results_train, df_pivot])
            
    # ============================================
    
    cr_test = clf_metrics(logit, X_test, y_test, label_metrica="teste", cutoff=cutoff)
    df_cr_test = pd.DataFrame(cr_test)
    
    # o .iloc[:-1, :] é pra excluir o "support" (pode trazer tb caso queira)
    df_melt = df_cr_test.iloc[:-1, :].reset_index(drop=False).melt(id_vars='index', var_name="type")
    df_melt['cutoff'] = cutoff

    # corrigindo os dados de acurácia
    accuracy = df_melt.query("type == 'accuracy'").copy()
    accuracy["index"] = "accuracy"

    df_melt = df_melt.drop(index=accuracy.index)
    df_melt = pd.concat([df_melt, accuracy.iloc[[0], :]]).reset_index(drop=False)

    # pivotando
    df_pivot = df_melt.pivot(columns=['type', 'index'], values='value', index='cutoff')
    
    df_results_test = pd.concat([df_results_test, df_pivot])
    
    # ============================================

In [ ]:
df_results_train

df_results_train.sort_values(('1', 'precision'), ascending=False)

In [ ]:
df_results_test

df_results_test.sort_values(('1', 'precision'), ascending=False)

In [ ]:
df_results_test

df_results_test.sort_values(('weighted avg', 'f1-score'), ascending=False)

_________

### Tradeoff precision/recall

Conforme é possível ver acima, claramente há um **tradeoff** entre precision e recall conforme variamos o cutoff. Isso faz total sentido, dado que estas métricas representam!

Podemos visualizar este tradeoff facilmente com o sklearn:

In [ ]:
y_proba_1 = logit.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, cutoffs = precision_recall_curve(y_test, y_proba_1)

Para plotar:

In [ ]:
plt.title("Precision-recall tradeoff")

plt.plot(recalls, precisions)

plt.xlabel("Recalls")
plt.ylabel("Precisons")

plt.show()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

PrecisionRecallDisplay.from_predictions(y_test, y_proba_1);

Ou, então:



In [ ]:
precisions.shape, recalls.shape, cutoffs.shape

In [ ]:
np.where(precisions == recalls)

In [ ]:
cutoffs[np.where(precisions == recalls)]

In [ ]:
plt.title("Precision-recall tradeoff")

plt.plot(cutoffs, precisions[:-1], label="precision")
plt.plot(cutoffs, recalls[:-1], label="recall")

plt.xlabel("Cutoffs")

ponto_de_encontro = cutoffs[np.where(precisions == recalls)]
plt.axvline(x=ponto_de_encontro, ls=":", color="k")

plt.legend()
plt.show()

________

### Curva ROC e AUC-ROC (AUROC)

Veremos agora uma outra métrica de avaliação de modelos de classificação que é intimamente ligada com os diferentes thresholds possíveis -- a **AUC (Area Under The Curve) da curva ROC (Receiver Operating Characteristics)**, por vezes chamada de **AUROC (Area Under the Receiver Operating Characteristics)**

A curva **ROC é uma curva de probabilidade**, sendo que **AUC é a área sob a curva**, representando **o grau de separabilidade atingido pelo modelo**.

Ou seja, esta medida nos diz **o quanto o modelo é capaz de distinguir entre duas classes**.

A curva ROC é construída com a **taxa de falsos positivos** no eixo x, e a **taxa de verdadeiros positivos** no eixo y, para diferentes **thresholds de classificação**:

<img src="https://miro.medium.com/max/1175/1*2nd7NTEBosPakccmLVWy9A.png" width=500>

O valor do AUC-ROC sempre estará **entre 0 e 1**, sendo que **quanto mais próximo de 1, melhor o modelo**.

> Valores de AUC-ROC maiores que 0.5 (mais próximos de 1) significam que o modelo tem uma **taxa de verdadeiros positivos maior que a taxa de falsos positivos**, ou seja, o modelo está acertando mais!

Quanto **mais próximo de 0** (para valores abaixo de 0.5), teremos um modelo que faz um bom trabalho em separar as classes, mas as classifica erroneamente.

E, quanto **mas próximo de 0.5**, pior é o modelo em separar as classes: seria um modelo que simplesmente chuta aleatoriamente ora a classe 0, ora a classe 1. 

Veja as imagens a seguir para uma ilustração:

<img src="https://miro.medium.com/max/528/1*Uu-t4pOotRQFoyrfqEvIEg.png" width=500>


<img src="https://miro.medium.com/max/507/1*yF8hvKR9eNfqqej2JnVKzg.png" width=500>


<img src="https://miro.medium.com/max/430/1*iLW_BrJZRI0UZSflfMrmZQ.png" width=500>


<img src="https://miro.medium.com/max/556/1*aUZ7H-Lw74KSucoLlj1pgw.png" width=500>

Ao olhar para a curva em si, temos a seguinte interpretação:

<img src="https://i.ytimg.com/vi/J9l8J1MeCbY/hqdefault.jpg" width=400>

Para aprender mais sobre a construção da curva ROC, sugiro [este StatQuest!](https://www.youtube.com/watch?v=4jRBRDbJemM)

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, cutoffs = roc_curve(y_test, y_proba_1)

Para plotar:

In [ ]:
plt.title("ROC curve")

plt.plot(fpr, tpr)

plt.xlabel("FPR")
plt.ylabel("TPR")

x = np.linspace(0, 1, 2)
plt.plot(x, x, color="k", ls=":")

plt.show()

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_test, y_proba_1)

x = np.linspace(0, 1, 2)
plt.plot(x, x, color="k", ls=":");

Por fim, pra calcular o AUC-ROC:

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_proba_1)

___
___
___

### E se tivermos uma classificação multiclasse?

Há problemas em que temos um problema de **classificação multiclasse**, pois há mais do que duas classes a serem preditas.

<img src="https://utkuufuk.com/2018/06/03/one-vs-all-classification/one-vs-all.png">

Boa noitícia: o operacional de construção e avaliação do modelo com o sklearn muda em absolutamente **nada**.

No entanto, conceitualmente, há algumas mudanças: a rigor, o modelo passa a se chamar **regresão logística MULTINOMIAL**, cujo processo de classificação é dado pela função **softmax**:

<img src="https://i.stack.imgur.com/YLeRi.png" width=600>

Para quem quiser saber mais sobre o "logit score", [clique aqui](https://stats.stackexchange.com/questions/329857/what-is-the-difference-between-decision-function-predict-proba-and-predict-fun).

Essencialmente, esse é o valor do termo linear usado como argumento da sigmoide, isto é, $z(x) = b_0 + b_1 x_1 + \cdots + b_p x_p$

É possível capturar o score pelo método `decision_function`:

In [ ]:
X_test.iloc[0]

In [ ]:
logit.intercept_, logit.coef_

In [ ]:
logit.intercept_ + (logit.coef_[0]*X_test.iloc[0].values).sum()

In [ ]:
logit.decision_function(X_test)

In [ ]:
1/(1 + np.exp(-(logit.intercept_ + (logit.coef_[0]*X_test.iloc[0].values).sum())))

In [ ]:
logit.predict_proba(X_test)

Na lista de exercícios, vocês trabalharão com um problema multiclasse (dataset `iris`). Não deixe de explorar mais a fundo o `decision_function` neste dataset, para entender seu funcionamento!